# Import data


In [1]:
import pandas as pd
import datetime as datetime
import matplotlib
import time

start_time= time.time()

dateTimeObj= time.time()
price_history= pd.read_excel(r'D:\TodaysPrice\Historic_database.xlsx', parse_dates= True)
price_history['Date']=pd.to_datetime(price_history.Date)
price_history= price_history.sort_values(by= ['Date'], ascending= True)

price_history_onlyOrdinary= price_history['Type'].isin(['Ordinary'])
price_history_onlyOrdinary= price_history[price_history_onlyOrdinary]
only_Debenture= price_history_onlyOrdinary['Sector'].isin(['Debenture'])
debenture_excluded= price_history_onlyOrdinary[~only_Debenture]
pivot_price= pd.pivot_table(price_history, values= 'Closing Price', 
                             index= ['Date'], columns= ['Ticker']).fillna(method='ffill')

# Sort companies

In [2]:
#Getting a lsit of companies
df= pivot_price
companies= price_history['Ticker'].unique()
company=companies.tolist()
company = [item for item in company if not(pd.isnull(item)) == True]
company.sort()


# Get list of dataframes

In [3]:
master=[]
for i in company:
    company_data= df[i]
    company_df= company_data.to_frame()
    company_df.columns= ['Closing Price']
    ma= [5,10,20,30,50]
    company_return=[]
    for day in ma:
        def SMA(df):
            sma_tr= df.copy()
            sma_tr['MA']= sma_tr['Closing Price'].rolling(day).mean()
            sma_tr['Ratio']= sma_tr['Closing Price']/ sma_tr['MA']
            sma_tr.loc[(sma_tr['Ratio']>1), 'Pos']= 1
            sma_tr.fillna(0, inplace=True)
            sma_tr['Return']= sma_tr['Closing Price'].pct_change(1)
            sma_tr['Value']= 1+sma_tr['Return']*sma_tr['Pos'].shift(1)
            sma_tr['Value'].fillna(1, inplace= True)
            sma_tr['Cumul']= sma_tr['Value'].cumprod()
            sma_tr[f'{i}']= 1*sma_tr['Cumul']
            final= sma_tr[[f'{i}']]
            final= final.copy()
            final['MA']=day
            return final
        final_return= SMA(company_df).iloc[-1]
        company_return.append(final_return)
    data= pd.DataFrame(company_return)
    data.set_index('MA', inplace=True)
    master.append(data)

# Obtain final matrix

In [4]:
final_product= master[0]
for num in range(len(company)-1):
    final_product= pd.merge(final_product, master[num+1], on='MA')
final_matrix= final_product.transpose()
final_matrix['Max'] = final_matrix.idxmax(axis =1)
final_matrix

MA,5.0,10.0,20.0,30.0,50.0,Max
ACLBSL,1.287743,1.208440,0.926861,0.878798,0.974687,5.0
ADBL,1.222989,1.098559,1.200139,1.318228,1.373060,50.0
ADBLD83,0.923420,0.927004,0.948623,0.946099,0.998157,50.0
AHPC,3.358512,2.486773,2.705571,2.101055,1.606912,5.0
AIL,1.330831,1.467455,1.300967,0.977143,1.015387,10.0
...,...,...,...,...,...,...
UPCL,2.398379,2.969164,3.514820,3.133206,3.643836,50.0
UPPER,2.027348,1.689813,1.622652,1.576110,1.427839,5.0
USLB,1.575117,1.305682,1.315262,0.925689,1.028203,5.0
VLBS,1.524309,1.414520,1.209732,1.165511,1.269664,5.0
